In [ ]:
import os
import cv2
import gradio as gr
import numpy as np
from art import tprint
from PIL import Image 
import matplotlib.pyplot as plt

###### **Жуки**

In [ ]:
path = "local/Yolo dataset/bug/"

###### <u>Создание обучающей выборки</u>

In [ ]:
extrapath = ["train", "val"][0]
ii = 9
for i in range(ii, ii + 1):
    im = Image.open(f"{path}{extrapath}/images/{i+1}.png")
    width, height = im.size
    xx, yy = [], []
    with open(f"{path}{extrapath}/labels/{extrapath[0]}_{i+1}.txt", 'r') as f:
        with open(f"{path}/{extrapath}/labels/{i+1}.txt", 'w') as ff:
            for line in f:
                xywh = line.strip().split()
                xx.append([float(xywh[1]) - float(xywh[3])/2, float(xywh[1]) + float(xywh[3])/2])
                yy.append([float(xywh[2]) - float(xywh[4])/2, float(xywh[2]) + float(xywh[4])/2])
                print(f"{i+1}: Центр [{float(xywh[1])}, {float(xywh[2])}], Диаметр: {float(xywh[3])}")
                ff.write(f"{xywh[0]} {float(xywh[1])/width} {float(xywh[2])/height} {float(xywh[3])/width} {float(xywh[4])/height}\n")
    plt.imshow(im)
    for j in range(len(xx)):
        for ii in [[0,0,0,1],[0,1,0,0],[0,1,1,1],[1,1,0,1]]:
            plt.plot([xx[j][ii[0]], xx[j][ii[1]]], [yy[j][ii[2]], yy[j][ii[3]]], c='r')
    plt.show()

###### <u>Обучение</u>

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

# Загрузка модели YOLOv8
model = YOLO(path+'myYOLO.pt')

# Список цветов для различных классов
colors = [
    (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255),
    (255, 0, 255), (192, 192, 192), (128, 128, 128), (128, 0, 0), (128, 128, 0),
    (0, 128, 0), (128, 0, 128), (0, 128, 128), (0, 0, 128), (72, 61, 139),
    (47, 79, 79), (47, 79, 47), (0, 206, 209), (148, 0, 211), (255, 20, 147)
]

In [ ]:
model.train(data=path + "fortrain.yaml", epochs=200)

In [ ]:
# model.save('myYOLObug.pt')

###### <u>Применение фреймворка</u>

In [ ]:
# Открытие исходного видеофайла
N_exp = 2
filename = f'{N_exp}.mp4'
capture = cv2.VideoCapture(path + 'fulls/' + filename)

# Чтение параметров видео
fps = int(capture.get(cv2.CAP_PROP_FPS))
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Настройка выходного файла
output_video_path = path + 'res/' + filename
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

count = 0 
boxes_in_frames = []
f = open(path + 'res/' + f"boxes_{N_exp}.txt", 'w')
while True:
    # Захват кадра
    ret, frame = capture.read()
    count += 1
    print(f"Кадр {count}")
    if not ret: # or count > 20:
        break

    # Обработка кадра с помощью модели YOLO
    results = model(frame)[0]

    # Получение данных об объектах
    boxes_in_frame = []
    classes_names = results.names
    classes = results.boxes.cls.cpu().numpy()
    boxes = results.boxes.xyxy.cpu().numpy().astype(np.int32)
    # Рисование рамок и подписей на кадре
    for class_id, box, conf in zip(classes, boxes, results.boxes.conf):
        if conf > 0.5:
            class_name = classes_names[int(class_id)]
            color = colors[int(class_id) % len(colors)]
            x1, y1, x2, y2 = box
            boxes_in_frame += [box]
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    boxes_in_frames += [boxes_in_frame]

    # Запись положения найденного тела
    n = len(boxes_in_frame)
    txt = f"{n} "
    for i in range(n):
        for j in range(4):
            txt += f"{boxes_in_frame[i][j]} "
    f.write(txt + "\n")

    # Запись обработанного кадра в выходной файл
    writer.write(frame)

# Освобождение ресурсов и закрытие окон
capture.release()
writer.release()
f.close()